# General Settings and Imports

In [153]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [154]:
#general stuff
import pandas as pd
import numpy as np
import json
from pprint import pprint

#bfs
from collections import deque

#ofac stuff
import csv
import requests
#import json
import xmltodict
from collections import defaultdict

#for graphsense tag data extraction
#from fastparquet import ParquetFile
#import snappy

#for flipside.xyz (SQL Databse)
#from flipside import Flipside

#graphsense stuff
import graphsense
from graphsense.api import addresses_api, bulk_api, entities_api, general_api, tags_api
from graphsense.model.neighbor_addresses import NeighborAddresses

#coinbase stuff
#import json, requests
import hmac, hashlib, time
from requests.auth import AuthBase

In [155]:
f = open('config.json')
config = json.load(f)
f.close()

CURRENCY = 'eth'
ADDRESS = '0x0Daa654baf0Caa38E89Bc675fFB591537E5D9840'

configuration = graphsense.Configuration(
    host = config['graphsense']['host'],
    api_key = {'api_key': config['graphsense']['api_key']})

#with graphsense.ApiClient(configuration) as api_client:
    #api_instance = general_api.GeneralApi(api_client)
    #api_response = api_instance.get_statistics()
    #pprint(api_response)

# Associated Wallets Module

In [156]:
class AssociatedWallets():
    
    def simple_neighbors(self, address, direction):
        with graphsense.ApiClient(configuration) as api_client:
            # Create an instance of the API class
            api_instance = addresses_api.AddressesApi(api_client)
            currency = CURRENCY # str | The cryptocurrency code (e.g., eth)
            address = ADDRESS # str | The cryptocurrency address
            direction = "out" # str | Incoming or outgoing neighbors
            only_ids = [
                "only_ids_example",
            ] # [str] | Restrict result to given set of comma separated addresses (optional)
            include_labels = False # bool | Whether to include labels of first page of address tags (optional) if omitted the server will use the default value of False
            page = "page_example" # str | Resumption token for retrieving the next page (optional)
            pagesize = 10 # int | Number of items returned in a single page (optional)

            # example passing only required values which don't have defaults set
            try:
                # Get an address's neighbors in the address graph
                api_response = api_instance.list_address_neighbors(currency, address, direction)
                return api_response
            except graphsense.ApiException as e:
                print("Exception when calling AddressesApi->list_address_neighbors: %s\n" % e)

            # example passing only required values which don't have defaults set
            # and optional values
            #try:
                # Get an address's neighbors in the address graph
                #api_response = api_instance.list_address_neighbors(currency, address, direction, only_ids=only_ids, include_labels=include_labels, page=page, pagesize=pagesize)
                #pprint(api_response)
            #except graphsense.ApiException as e:
                #print("Exception when calling AddressesApi->list_address_neighbors: %s\n" % e)

    #Utility function to get neighbors of address        
    def get_addr_neighbors(self, address, direction):

        if (direction == 'out'):
            degree = 'out_degree'
            address_degree = 'address_out_degree'
        else:
            degree = 'in_degree'
            address_degree = 'address_in_degree'

        with graphsense.ApiClient(configuration) as api_client:
            try:
                api_instance = bulk_api.BulkApi(api_client)
                print(f"get_addr_neighbors of {address}")
                operation = "list_address_neighbors"
                body = {'address': [address], 'direction': direction}

                df_address_neighbors = pd.read_csv(api_instance.bulk_csv(CURRENCY, operation, body=body,
                                                                    num_pages=1, _preload_content=False))
                
                df_address_neighbors = df_address_neighbors \
                    .loc[(df_address_neighbors['_error'] != 'not found') &
                        (df_address_neighbors['_info'] != 'no data')].reset_index(drop=True)
                
                if df_address_neighbors.empty:
                    df_address_neighbors.columns = ['address', 'entity', degree]
                    return df_address_neighbors
                
                print(df_address_neighbors.columns)  # Print column names

                
                df_address_neighbors = df_address_neighbors \
                    .rename(columns={'address_address': 'address', 
                                    'address_entity': 'entity',
                                    address_degree: degree})

                return df_address_neighbors[['address', 'entity', degree]]
                
            except graphsense.ApiException as e:
                print("Exception when calling Bulk Api: %s\n" % e)

    # The Breadth-First Search algorithm:
    def bfs_neighbors(self, seed_address, max_depth, max_degree, verbose, direction):

        if (direction == 'out'):
            degree = 'out_degree'
            #address_degree = 'address_out_degree'
        else:
            degree = 'in_degree'
            #address_degree = 'address_in_degree'
        
        # collect neighbors
        neighbors = []
        for i in range (0, max_depth):
            neighbors.insert(i, [])
        neighbors[0].append(seed_address)

        #keep track of depth
        levels = {seed_address: 0}
                
        # record visited addresses and entities
        visited_addresses = set([seed_address])
        
        # maintain a queue of addresses
        queue = deque([(seed_address, 0)])

        while(queue):

            # get first address from the queue
            addr, level = queue.popleft()            

            # retrieve address neighbors
            df_neighbors = self.get_addr_neighbors(addr, direction)

            # continue with neighbors out_degree < max_outdegree
            for index, neighbor in df_neighbors.iterrows():

                 # stop if address has already been visited
                if(neighbor['address'] in visited_addresses):
                    if verbose:
                        print(addr, end=' ') 
                        print("STOP | same address")
                    continue
                                
                # stop if max depth is reached
                if level + 1 == max_depth:
                    print(level)
                    if verbose:
                        print(addr, end=' ') 
                        print("STOP | max depth")
                    continue

                neighbors[level+1].append(neighbor['address'])

                # stop if address out_degree exceeds threshold
                if(neighbor[degree] > max_degree):
                    if verbose:
                        print(addr, end=' ') 
                        print("STOP | max degree")
                    continue
                
                queue.append((neighbor['address'], level + 1))
                visited_addresses.add(neighbor['address'])
                levels[neighbor['address']] = level + 1
                    
            if len(queue) == 0:
                return neighbors

# Blacklist Module

In [157]:
class Blacklist():

    def ofac(self):

        coins = []
        myCoins = {}
        sdn = defaultdict(list)
        filename = 'sdn2.xml'
        URL = "https://www.treasury.gov/ofac/downloads/sanctions/1.0/sdn_advanced.xml"

        response = requests.get(URL)
        with open(filename, 'wb') as file:
            file.write(response.content)

        xml_data = open(filename, 'r').read()  # Read file
        d = xmltodict.parse(xml_data)

        for i in d['Sanctions']['ReferenceValueSets']['FeatureTypeValues']['FeatureType']:
            if 'Digital Currency Address' in i['#text']:
                coins.append(i['@ID'])
                myCoins[i['@ID']] =  i['#text'].replace('Digital Currency Address - ','')
            #   print(i['@ID'],'-',i['#text'])

        for i in d['Sanctions']['DistinctParties']['DistinctParty']:
            if 'Feature' in i['Profile'].keys():
                for j in i['Profile']['Feature']:
                    if '@FeatureTypeID' in j:
                        if type(j) is not str:
                            if str(j['@FeatureTypeID']) in coins:
                            #   print(j)
                            #   print(j['FeatureVersion']['VersionDetail']['#text'])
                                sdn[myCoins[j['@FeatureTypeID']]].append(j['FeatureVersion']['VersionDetail']['#text'])            
                            #    break
            
        with open('results/sdn.json', 'w') as fp:
            json_data = json.dump(sdn, fp)
        fp.close()

    def check_membership(self, currency, address, json_data):

        currency_set = set(json_data[currency])

        return address in currency_set
    
    def dark_web(self):

        filename = "/Users/sticky/Documents/Bachelor_Thesis/data/address_cluster.parquet/address_id_group=2/part-00520-b3ffb0c8-9bc9-45d0-b89b-a8f63c16284b.c000.snappy.parquet"
        def snappy_decompress(data, uncompressed_size):
            return snappy.decompress(data)
        pf = ParquetFile('filename') # filename includes .snappy.parquet extension
        df = pf.to_pandas()
        print(df)



# Exchanges Module

In [158]:
class Exchange():
    
    def __init__(self, depth, direction):
        self.DEPTH = depth
        self.DIRECTION = direction

    def get_ex_tags(self, address, type):

        # Enter a context with an instance of the API client
        with graphsense.ApiClient(configuration) as api_client:
            # Create an instance of the API class
            api_instance = addresses_api.AddressesApi(api_client)

            currency = CURRENCY # str | The cryptocurrency code (e.g., btc)
            #address = ADDRESS # str | The cryptocurrency address
            page = "page_example" # str | Resumption token for retrieving the next page (optional)
            pagesize = 10 # int | Number of items returned in a single page (optional)
            include_tags = True

            # example passing only required values which don't have defaults set
            try:
                # Get attribution tags for a given address
                api_response = api_instance.list_tags_by_address(currency, address.lower())
                exchange_data = api_response
                
                exchange_tags = []

                for entry in exchange_data['address_tags']:
                    actor = entry.get('actor', '')
                    category = entry.get('category', '')
                    label = entry.get('label', '')
                    if ((actor or label) and (type == 'cex' and (category == 'exchange' or category == 'market'))) or ((actor or label) and type == 'dex' and category == 'defi_dex'):
                        exchange_tags.append({
                            'actor': actor,
                            'category': category,
                            'currency': entry.get('currency', ''),
                            'label': label
                        })

                return exchange_tags
                
            except graphsense.ApiException as e:
               print("Exception when calling AddressesApi->list_tags_by_address: %s\n" % e)
    
    def get_ex_used(self, type):

        #hier einfach neighbors von ADDRESS finden mit der self.depth dann jedes von denen in get_cex_tags und dann falls result nicht empty ist einf in einer liste alles speichern und dann printen
        
        ass_wallets = AssociatedWallets()
        max_degree = 7
        verbose = True
        neighbors = ass_wallets.bfs_neighbors(ADDRESS, self.DEPTH, max_degree, verbose, self.DIRECTION)

        exchanges_used = []

        for arr_level in neighbors:
            for address in arr_level:
                temp_exchange_tags = self.get_ex_tags(address, type)
                if temp_exchange_tags:
                    exchanges_used.append(temp_exchange_tags)

        return exchanges_used

# Proof of Source Module

In [159]:
# Create custom authentication for Coinbase API
class CoinbaseWalletAuth(AuthBase):
    def __init__(self, api_key, secret_key):
        self.api_key = api_key
        self.secret_key = secret_key

    def __call__(self, request):
        timestamp = str(int(time.time()))
        message = timestamp + request.method + request.path_url + (request.body or '')
        signature = hmac.new(self.secret_key, message, hashlib.sha256).hexdigest()

        request.headers.update({
            'CB-ACCESS-SIGN': signature,
            'CB-ACCESS-TIMESTAMP': timestamp,
            'CB-ACCESS-KEY': self.api_key,
        })
        return request

class PoS():
    
    def coinbase():

        # Before implementation, set environmental variables with the names API_KEY and API_SECRET

        f = open('config.json')
        config = json.load(f)
        f.close()   

        API_KEY = config['coinbase']['api_key']
        API_SECRET = config['coinbase']['api_secret']
        api_url = config['coinbase']['host']
        auth = CoinbaseWalletAuth(API_KEY, API_SECRET)

        # Get current user
        r = requests.get(api_url + 'user', auth=auth)
        print(r.json())
        # {u'data': {u'username': None, u'resource': u'user', u'name': u'User'...

        # Send funds
        tx = {
            'type': 'send',
            'to': 'user@example.com',
            'amount': '10.0',
            'currency': 'USD',
        }
        r = requests.post(api_url + 'accounts/primary/transactions', json=tx, auth=auth)
        print(r.json())
        # {u'data': {u'status': u'pending', u'amount': {u'currency': u'BTC'...

# Main Section

In [160]:
# Function calls

#ass_wallets = AssociatedWallets(CURRENCY)

#direction = 'out'

#addr_neigh = ass_wallets.bfs_neighbors(ADDRESS, 4, 10, True, direction)
#addr_simple_neigh = ass_wallets.simple_neighbors(ADDRESS, direction)

#with open("results/addresses_" + direction + ".txt", "w") as outfile:
    #outfile.write(str(addr_neigh))

#blacklist = Blacklist()
#blacklist.ofac()
#print(blacklist.check_membership('ETH', ADDRESS, json_data))
#blacklist.dark_web()

 # Open the JSON file
#with open('results/sdn.json', 'r') as file:
    # Load the JSON data
    #json_data = json.load(file)


ex = Exchange(2, 'out')
exchanges_used = ex.get_ex_used('dex')
#exchanges_used = ex.get_ex_tags('0x12459C951127e0c374FF9105DdA097662A027093', 'dex')
pprint(exchanges_used)


get_addr_neighbors of 0x0Daa654baf0Caa38E89Bc675fFB591537E5D9840
Index(['_error', '_info', '_request_address', 'address_actors',
       'address_actors_count', 'address_address', 'address_balance_eur',
       'address_balance_usd', 'address_balance_value', 'address_currency',
       'address_entity', 'address_first_tx_height',
       'address_first_tx_timestamp', 'address_first_tx_tx_hash',
       'address_in_degree', 'address_is_contract', 'address_last_tx_height',
       'address_last_tx_timestamp', 'address_last_tx_tx_hash',
       'address_no_incoming_txs', 'address_no_outgoing_txs',
       'address_out_degree', 'address_status', 'address_token_balances',
       'address_token_balances_usdc_eur', 'address_token_balances_usdc_usd',
       'address_token_balances_usdc_value', 'address_token_balances_usdt_eur',
       'address_token_balances_usdt_usd', 'address_token_balances_usdt_value',
       'address_token_balances_weth_eur', 'address_token_balances_weth_usd',
       'address_toke